
# TP Pickle Explorer

This notebook helps you inspect and visualize the contents of `bobcat_to_diamondback.pickle`, and export slices for downstream use (e.g., PICASO-ready profiles after you identify which axis corresponds to pressure/temperature).

**What it does:**
1. Loads the pickle (a NumPy array).
2. Prints shape and basic stats.
3. Plots variable curves vs. the vertical index for selected (Teff, g) indices.
4. Shows mean curves over (Teff, g) for each variable.
5. Lets you export a chosen (Teff, g) and variable index as CSV.
6. (Optional) Demonstrates interpolation to a uniform log-pressure grid once you identify which variable is pressure and which is temperature.


In [ ]:

# --- 0) Paths
from pathlib import Path

PICKLE_PATH = Path("/mnt/data/bobcat_to_diamondback.pickle")  # update here if needed
assert PICKLE_PATH.exists(), f"File not found: {PICKLE_PATH}"
PICKLE_PATH


In [ ]:

# --- 1) Load
import numpy as np

arr = np.load(PICKLE_PATH, allow_pickle=True)
arr.shape



**Axis convention (tentative):**
- Axis 0: Teff index (size ~301)
- Axis 1: Gravity index (size ~41)
- Axis 2: Variable index (size 5)
- Axis 3: Vertical levels (size ~91)

> The file does not carry labels. We'll visualize and you can map variables to physical quantities (e.g., pressure, temperature, scaling factors) using domain knowledge or advisor notes.


In [ ]:

# --- 2) Basic stats by variable
def describe_component(a):
    return {
        "min": float(np.nanmin(a)),
        "max": float(np.nanmax(a)),
        "finite_frac": float(np.isfinite(a).mean())
    }

nT, nG, nV, nL = arr.shape
stats = [describe_component(arr[:,:,iv,:]) for iv in range(nV)]
stats


In [ ]:

# --- 3) Quick plots: variables vs. vertical index for selected (Teff, g)
import matplotlib.pyplot as plt

examples = [(0,0), (nT//2, 0), (nT//2, nG//2), (nT-1, nG-1)]

for (iT, iG) in examples:
    plt.figure()
    for iv in range(nV):
        plt.plot(range(nL), arr[iT, iG, iv, :], label=f"var{iv}")
    plt.xlabel("Vertical index (0..nL-1)")
    plt.ylabel("Value")
    plt.title(f"Slices at iT={iT}, iG={iG}")
    plt.legend()
    plt.show()


In [ ]:

# --- 4) Mean over (Teff, g) for each variable vs. vertical index
for iv in range(nV):
    plt.figure()
    mean_curve = np.mean(arr[:, :, iv, :], axis=(0,1))
    plt.plot(range(nL), mean_curve, label=f"mean var{iv}")
    plt.xlabel("Vertical index")
    plt.ylabel("Mean value over (Teff, g)")
    plt.title(f"Mean curve over (Teff, g): var{iv}")
    plt.legend()
    plt.show()



## 5) Export a (Teff, g, var) slice as CSV

Pick indices and export the vertical profile for that variable to CSV for offline inspection.


In [ ]:

# Set these indices and variable to export:
iT_sel = nT//2   # e.g., mid Teff index
iG_sel = nG//2   # e.g., mid gravity index
iv_sel = 0       # variable index 0..(nV-1)

out_csv = Path("/mnt/data/tp_pickle_slice_T{}_G{}_V{}.csv".format(iT_sel, iG_sel, iv_sel))
np.savetxt(out_csv, arr[iT_sel, iG_sel, iv_sel, :], delimiter=",")
out_csv



## 6) (Optional) Interpolate onto a uniform log-pressure grid

Once you identify which variable corresponds to **pressure** (monotonic with level) and which corresponds to **temperature**, you can use the function below. Fill in `pressure_vals` and `temperature_vals` accordingly.


In [ ]:

import numpy as np

def interpolate_to_uniform_logP(P_bar, T_K, n_levels=100, p_min=None, p_max=None):
    P = np.asarray(P_bar).astype(float)
    T = np.asarray(T_K).astype(float)
    mask = np.isfinite(P) & np.isfinite(T) & (P > 0)
    P, T = P[mask], T[mask]
    order = np.argsort(P)
    P, T = P[order], T[order]

    pmin = P.min() if p_min is None else max(p_min, P.min())
    pmax = P.max() if p_max is None else min(p_max, P.max())
    logP = np.log10(P)
    grid = np.linspace(np.log10(pmin), np.log10(pmax), n_levels)
    T_new = np.interp(grid, logP, T)
    P_new = 10**grid
    return P_new, T_new

# Example (placeholder: replace with real mappings)
# Suppose varX is pressure (in bars) and varY is temperature (K):
# pressure_vals = arr[iT_sel, iG_sel, varX, :]
# temperature_vals = arr[iT_sel, iG_sel, varY, :]
# Pn, Tn = interpolate_to_uniform_logP(pressure_vals, temperature_vals, n_levels=120)
# Pn[:5], Tn[:5]
